In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
import csv
import pandas as pd

hostip = "192.168.0.140"

def batch_get(f,size):
    rw = sum(1 for h in open(f)) 
    no_r = random.sample(range(1,rw),(rw-1)-size)
    r_sort = sorted(no_r)
    result = pd.read_csv(f, skiprows=r_sort)
    return result

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
    fname = 'click_stream_rt.csv'
    producer = connect_kafka_producer()
    while True:
        size = random.randint(500, 1000)
        # getting the batch at rondom
        b = batch_get(fname, size)
        part_b = len(b) // 5 # getting the partion of batch to add timestamp to five sets
        parts = [] # List to store partitions
        q = 0 # Counter for partitions
        for i in range(0, len(b), part_b):
            if q != 4:
                part = b[i:i + part_b].reset_index(drop=True)# Create partitions with equal size (except the last one)
                parts.append(part)
            else:
                part_e = len(b) - (part_b*4) # For the last partition, consider the remaining elements
                part = b[i:i + part_e].reset_index(drop=True)
                parts.append(part)
                break;
            q = q + 1 # Increment the partition counter
        ts = int(dt.datetime.now().timestamp()) # Get current timestamp
        e = 0 # Counter for timestamps
        for p in parts:
            p['ts'] = ts + e # Add timestamp to each partition
            e = e + 1 # Increment the timestamp counter
        latest = pd.concat(parts) # adding partitions for updating the batch
        latest.reset_index(drop=True, inplace=True)# Reseting index for the updated batch
        latestb = latest.to_dict(orient='records') # Convert the updated batch to a list of dicts
        print("Transfering batch of size:"+str(len(b))+"rows")
        publish_message(producer, "Stream_Data_MOTH", latestb)#Reseting idx for updated batch
        sleep(5) #waiting five secs to send nxt btch

Transfering batch of size: 969 rows
Transfering batch of size: 677 rows
Transfering batch of size: 837 rows
Transfering batch of size: 712 rows
Transfering batch of size: 623 rows
Transfering batch of size: 940 rows
Transfering batch of size: 578 rows
Transfering batch of size: 871 rows
Transfering batch of size: 945 rows
Transfering batch of size: 504 rows
Transfering batch of size: 792 rows
Transfering batch of size: 745 rows
Transfering batch of size: 837 rows
Transfering batch of size: 701 rows
Transfering batch of size: 733 rows
Transfering batch of size: 733 rows
Transfering batch of size: 941 rows
Transfering batch of size: 955 rows
Transfering batch of size: 881 rows
Transfering batch of size: 578 rows
Transfering batch of size: 633 rows
Transfering batch of size: 850 rows
Transfering batch of size: 803 rows
Transfering batch of size: 688 rows
Transfering batch of size: 926 rows
Transfering batch of size: 807 rows
Transfering batch of size: 767 rows
Transfering batch of size: 7